
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets


## Parsing JSON Data

In [0]:
%sql
SELECT *
FROM customers
LIMIT 3

customer_id,email,profile,updated
C00001,dabby2y@japanpost.jp,"{""first_name"":""Dniren"",""last_name"":""Abby"",""gender"":""Female"",""address"":{""street"":""768 Mesta Terrace"",""city"":""Annecy"",""country"":""France""}}",2021-12-14T23:15:43.375Z
C00002,eabbysc1@github.com,"{""first_name"":""Etti"",""last_name"":""Abbys"",""gender"":""Female"",""address"":{""street"":""1748 Vidon Plaza"",""city"":""Varge Mondar"",""country"":""Portugal""}}",2021-12-14T23:15:43.375Z
C00003,rabelovd1@wikispaces.com,"{""first_name"":""Ronnie"",""last_name"":""Abelov"",""gender"":""Male"",""address"":{""street"":""363 Randy Park"",""city"":""San Celestio"",""country"":""Philippines""}}",2021-12-14T23:15:43.375Z


In [0]:
%sql
DESCRIBE customers

col_name,data_type,comment
customer_id,string,null
email,string,null
profile,string,null
updated,string,null


In [0]:
%sql
SELECT customer_id, profile:first_name, profile:address:country 
FROM customers
LIMIT 3

customer_id,first_name,country
C00001,Dniren,France
C00002,Etti,Portugal
C00003,Ronnie,Philippines


In [0]:
%sql
-- this query will be failed due to profile field is string and does not have json structure
SELECT from_json(profile) AS profile_struct
FROM customers;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [WRONG_NUM_ARGS.WITH_SUGGESTION] Invalid number of arguments for the function `from_json`. Consider to change the number of arguments because the function requires [2, 3] parameters but the actual number is 1.; line 1 pos 7
	at org.apache.spark.sql.errors.QueryCompilationErrors$.invalidFunctionArgumentsError(QueryCompilationErrors.scala:672)
	at org.apache.spark.sql.errors.QueryCompilationErrors$.invalidFunctionArgumentNumberError(QueryCompilationErrors.scala:687)
	at org.apache.spark.sql.catalyst.analysis.FunctionRegistryBase$.$anonfun$build$7(FunctionRegistry.scala:152)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.analysis.FunctionRegistryBase$.$anonfun$build$4(FunctionRegistry.scala:147)
	at org.apache.spark.sql.catalyst.analysis.FunctionRegistry$.$anonfun$expression$1(FunctionRegistry.scala:989)
	at org.apache.spark.sql.catalyst.analysis.SimpleFunctionRegistryBase.lookupFunction(FunctionRegistry.scala:243)
	at org.apache.spark.sql.catalyst.analysis.SimpleFunctionRegistryBase.lookupFunction$(FunctionRegistry.scala:237)
	at org.apache.spark.sql.catalyst.analysis.SimpleFunctionRegistry.lookupFunction(FunctionRegistry.scala:309)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalogImpl.$anonfun$resolveBuiltinOrTempFunctionInternal$1(SessionCatalog.scala:2641)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalogImpl.lookupTempFuncWithViewContext(SessionCatalog.scala:2651)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalogImpl.resolveBuiltinOrTempFunctionInternal(SessionCatalog.scala:2641)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalogImpl.resolveBuiltinOrTempFunction(SessionCatalog.scala:2618)
	at org.apache.spark.sql.catalyst.catalog.DelegatingSessionCatalog.resolveBuiltinOrTempFunction(DelegatingSessionCatalog.scala:488)
	at org.apache.spark.sql.catalyst.catalog.DelegatingSessionCatalog.resolveBuiltinOrTempFunction$(DelegatingSessionCatalog.scala:485)
	at com.databricks.sql.managedcatalog.ManagedCatalogSessionCatalog.resolveBuiltinOrTempFunction(ManagedCatalogSessionCatalog.scala:72)
	at com.databricks.sql.analyzer.UnresolvedFunctionLogging.$anonfun$resolveBuiltinOrTempFunction$1(UnresolvedFunctionLogging.scala:80)
	at com.databricks.sql.analyzer.UnresolvedFunctionLogging.recordFailure(UnresolvedFunctionLogging.scala:97)
	at com.databricks.sql.analyzer.UnresolvedFunctionLogging.resolveBuiltinOrTempFunction(UnresolvedFunctionLogging.scala:80)
	at com.databricks.sql.analyzer.UnresolvedFunctionLogging.resolveBuiltinOrTempFunction$(UnresolvedFunctionLogging.scala:78)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFunctions$.resolveBuiltinOrTempFunction(Analyzer.scala:2976)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFunctions$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveFunctions$$resolveBuiltinOrTempFunction(Analyzer.scala:3175)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFunctions$$anonfun$apply$23$$anonfun$applyOrElse$165.$anonfun$applyOrElse$170(Analyzer.scala:3119)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:110)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFunctions$$anonfun$apply$23$$anonfun$applyOrElse$165.applyOrElse(Analyzer.scala:3119)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFunctions$$anonfun$apply$23$$anonfun$applyOrElse$165.applyOrElse(Analyzer.scala:3088)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:519)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:106)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:519)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$3(TreeNode.scala:524)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1323)
	at 

In [0]:
%sql
SELECT profile 
FROM customers 
LIMIT 1

profile
"{""first_name"":""Dniren"",""last_name"":""Abby"",""gender"":""Female"",""address"":{""street"":""768 Mesta Terrace"",""city"":""Annecy"",""country"":""France""}}"


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_customers AS
  SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')) AS profile_struct
  FROM customers;
  
SELECT * FROM parsed_customers LIMIT 3

customer_id,profile_struct
C00001,"List(List(Annecy, France, 768 Mesta Terrace), Dniren, Female, Abby)"
C00002,"List(List(Varge Mondar, Portugal, 1748 Vidon Plaza), Etti, Female, Abbys)"
C00003,"List(List(San Celestio, Philippines, 363 Randy Park), Ronnie, Male, Abelov)"


In [0]:
%sql
DESCRIBE parsed_customers

col_name,data_type,comment
customer_id,string,null
profile_struct,"struct,first_name:string,gender:string,last_name:string>",null


In [0]:
%sql
SELECT customer_id, profile_struct.first_name, profile_struct.address.country
FROM parsed_customers
LIMIT 3

customer_id,first_name,country
C00901,Gregoor,Sweden
C00902,Pearla,China
C00903,Parker,Poland


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW customers_final AS
  SELECT customer_id, profile_struct.*
  FROM parsed_customers;
  
SELECT *
FROM customers_final
LIMIT 3

customer_id,address,first_name,gender,last_name
C00901,"List(Trelleborg, Sweden, 0 Superior Park)",Gregoor,Male,Lenard
C00902,"List(Zengtian, China, 75 Dottie Way)",Pearla,Female,Lengthorn
C00903,"List(Jastrzębia, Poland, 70 Badeau Lane)",Parker,Male,Lequeux


In [0]:
%sql
SELECT order_id, customer_id, books
FROM orders
LIMIT 3

order_id,customer_id,books
000000000003559,C00001,"List(List(B09, 2, 48))"
000000000004243,C00002,"List(List(B07, 1, 33), List(B06, 1, 22))"
000000000004321,C00003,"List(List(B04, 2, 40))"


## Explode Function

In [0]:
%sql
SELECT order_id, customer_id, explode(books) AS book 
FROM orders
LIMIT 3

order_id,customer_id,book
000000000003559,C00001,"List(B09, 2, 48)"
000000000004243,C00002,"List(B07, 1, 33)"
000000000004243,C00002,"List(B06, 1, 22)"


## Collecting Rows

In [0]:
%sql
SELECT customer_id,
  collect_set(order_id) AS orders_set,
  collect_set(books.book_id) AS books_set
FROM orders
GROUP BY customer_id
LIMIT 3

customer_id,orders_set,books_set
C00001,"List(000000000005191, 000000000003559, 000000000005067)","List(List(B08, B02), List(B09), List(B03, B12))"
C00002,"List(000000000005192, 000000000004550, 000000000004243)","List(List(B04, B06), List(B02, B06, B01), List(B07, B06))"
C00003,"List(000000000004321, 000000000004575, 000000000005193)","List(List(B04, B10), List(B09, B06), List(B04))"



##Flatten Arrays

In [0]:
%sql
SELECT customer_id,
  collect_set(books.book_id) As before_flatten,
  array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
FROM orders
GROUP BY customer_id
LIMIT 3

customer_id,before_flatten,after_flatten
C00001,"List(List(B08, B02), List(B09), List(B03, B12))","List(B08, B02, B09, B03, B12)"
C00002,"List(List(B04, B06), List(B02, B06, B01), List(B07, B06))","List(B04, B06, B02, B01, B07)"
C00003,"List(List(B04, B10), List(B09, B06), List(B04))","List(B04, B10, B09, B06)"



##Join Operations

In [0]:
%sql
CREATE OR REPLACE VIEW orders_enriched AS
SELECT *
FROM (
  SELECT *, explode(books) AS book 
  FROM orders) o
INNER JOIN books b
ON o.book.book_id = b.book_id;

SELECT *
FROM orders_enriched
LIMIT 3

order_id,order_timestamp,customer_id,quantity,total,books,book,book_id,title,author,category,price
000000000003559,1657722056,C00001,2,48,"List(List(B09, 2, 48))","List(B09, 2, 48)",B09,Advanced Data Structures,Peter Brass,Computer Science,24.0
000000000004243,1658786901,C00002,2,55,"List(List(B07, 1, 33), List(B06, 1, 22))","List(B07, 1, 33)",B07,The Hundred-Page Machine Learning,Andriy Burkov,Computer Science,33.0
000000000004243,1658786901,C00002,2,55,"List(List(B07, 1, 33), List(B06, 1, 22))","List(B06, 1, 22)",B06,Deep Learning with Python,François Chollet,Computer Science,22.0


## Set Operations

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_updates
AS SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

SELECT * FROM orders 
UNION 
SELECT * FROM orders_updates 
LIMIT 3

order_id,order_timestamp,customer_id,quantity,total,books
000000000004362,1659003803,C00277,2,58,"List(List(B04, 1, 20), List(B11, 1, 38))"
000000000003709,1658002064,C00532,2,77,"List(List(B05, 1, 47), List(B12, 1, 30))"
000000000003737,1658042571,C00552,2,42,"List(List(B04, 1, 20), List(B06, 1, 22))"


In [0]:
%sql
SELECT * FROM orders 
INTERSECT 
SELECT * FROM orders_updates 
LIMIT 3

order_id,order_timestamp,customer_id,quantity,total,books


In [0]:
%sql
SELECT * FROM orders 
MINUS 
SELECT * FROM orders_updates 
LIMIT 2

order_id,order_timestamp,customer_id,quantity,total,books
000000000003559,1657722056,C00001,2,48,"List(List(B09, 2, 48))"
000000000004243,1658786901,C00002,2,55,"List(List(B07, 1, 33), List(B06, 1, 22))"


## Reshaping Data with Pivot

In [0]:
%sql
CREATE OR REPLACE TABLE transactions AS

SELECT * FROM (
  SELECT
    customer_id,
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (
  sum(quantity) FOR book_id in (
    'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
    'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);

SELECT * FROM transactions LIMIT 3

customer_id,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12
C00394,1,null,null,null,null,null,2,1,1,1,null,1
C00169,null,null,null,2,null,null,1,null,2,null,1,null
C00049,null,1,null,null,null,1,2,1,2,null,null,null
